In [118]:
import numpy as np
import pandas as pd
target_user_id=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/target_user_id.csv')
validation_test_data=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/validation_testing_data.csv')
final_final=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/final_final.csv')
df_news_all=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/df_news_all.csv')

from scipy import spatial
from tqdm import tqdm_notebook

import re
import os
import jieba
import jieba.posseg as pseg
import pickle

from gensim.models import Doc2Vec, doc2vec
from gensim.models import word2vec
import random
import logging

from itertools import zip_longest

In [119]:
bert_predict_output=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/project-BERt/bert/tmp/news_output/test_output/test_results.tsv',sep='\t')
bert_test_data_with_target = pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/bert_test_data_with_target.csv')
#取得變成columns name的資料
temp=bert_predict_output.columns.values

#轉成dataFrame
d = {'notClick':[temp[0]], 'Click':[temp[1]]} 
first_df=pd.DataFrame(d)

#將原本的欄位重新命名
bert_predict_output.columns=['notClick','Click']

#把兩個資料合併，注意 concat的第一個資料集須放變成欄位名稱的資料，不然位置會錯
bert_predict_output=pd.concat([first_df, bert_predict_output],ignore_index=True)
bert_predict_output.head()

#把整理好的資料再跟bert_test_data水平合併
bert_result = pd.concat([bert_test_data_with_target, bert_predict_output],axis=1)
bert_result = bert_result.drop(['label'],axis=1)

In [120]:
bert_result[["notClick", "Click"]] = bert_result[["notClick", "Click"]].apply(pd.to_numeric)

In [121]:
# # 這裡暫不使用
# target_user_id=target_csv.iloc[0,:].values[0]
# pred_time = validation_test_data[validation_test_data.user_id == target_user_id].iloc[0,:].time

# bert_result_date = bert_result.date
# for i in range(len(bert_result)):
#     if (pred_time-bert_result_date[i])>200000000: # 若該新聞發布日期和推薦時間超過兩個月則刪除該候選新聞
#         bert_result = bert_result.drop([i])
# bert_result = bert_result.reset_index(drop=True)

In [122]:
bert_result = bert_result.sort_values(by="Click" , ascending=False)
bert_result = bert_result.reset_index(drop=True)

In [123]:
bert_result.head(2)

,path,date,news_guid,title,news_content,content_remove_html_tag,notClick,Click
0,../datasets/news/2018-05-09/7853913e-eaf7-4c8d...,20180509085858,7853913E-EAF7-4C8D-A21F-2281A02E5DA5,精華Q1 EPS 7元；新產能陸續投產為今年營運添動能 ...,\n<P>MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導<...,MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導隱形眼鏡大廠...,0.407336,0.592664
1,../datasets/news/2018-02-01/4cbfc6ab-2526-4b87...,20180201133759,4CBFC6AB-2526-4B87-85BE-7B646A214BA7,環球晶：半導體矽晶圓仍有漲價空間，今年拚逐季成長 ...,\n<P>MoneyDJ新聞 2018-02-01 13:37:59 記者 新聞中心 報導<...,MoneyDJ新聞 2018-02-01 13:37:59 記者 新聞中心 報導半導體矽晶圓...,0.408350,0.591650


In [124]:
len(bert_result)

1651

In [125]:
#################### 選擇分數最高的前 n 筆
n = 1000
####################
bert_result = bert_result.drop(bert_result.index.values[(bert_result.index.values>n-1).tolist()].tolist())

In [126]:
target_user_id_str=target_user_id.iloc[0,:].values[0]
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if bert_result[bert_result.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')

else:
    print('目標已存在候選集合當中')

目標已存在候選集合當中


## 將最後一筆觀看紀錄加入資料當中

In [127]:
final_final.head(2)

,time,user_id,news_id,C011010,C011011,C011020,C011021,C011022,C011023,C011024,...,C099391,C099392,C099393,C099394,C099395,C099396,C099397,C099400,C099410,C099411
0,20180516222843,00039bc1-c70c-404d-9a16-5ec2227061f7,67172077-4536-405A-B5FD-E7793171C5E8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20180516223042,00039bc1-c70c-404d-9a16-5ec2227061f7,8C789423-6574-4BAA-A50A-7B3F9C25C8B8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
final_final = final_final.loc[:,'time':'news_id']

In [129]:
# 找出該 user 最後一筆看的 news
particular_user_last_news_id = final_final[final_final.user_id == target_user_id_str].tail(1).news_id.values[0]

In [130]:
df_news_all.head(2)

,path,date,news_guid,title,news_content,content_remove_html_tag
0,../datasets/news/1999-06-14/0b7a5d15-7bcd-11d3...,19990614074852,0B7A5D15-7BCD-11D3-98FF-00E018A00403,導線架市場重新洗牌新台和順德談策略聯盟 ...,\n\n\n<P>在大廠壓縮小廠生存空間，小廠試圖尋求同業策略聯盟的趨勢下，國內3大導線架廠...,在大廠壓縮小廠生存空間，小廠試圖尋求同業策略聯盟的趨勢下，國內3大導線架廠包括順德、佳茂、旭...
1,../datasets/news/2009-08-06/3b508922-e6a5-4c25...,20090806111315,3B508922-E6A5-4C25-B80F-01E8B36BB835,建築鋼每噸利潤達1000元 中國鐵礦石庫存逼新高 ...,\n<P>精實新聞 2009-08-06 11:13:15 記者 余美慧 報導</P><P>...,精實新聞 2009-08-06 11:13:15 記者 余美慧 報導聯合金屬網統計，截至7月...


In [131]:
particular_user_last_news = df_news_all[df_news_all.news_guid == particular_user_last_news_id]

In [132]:
particular_user_last_news

,path,date,news_guid,title,news_content,content_remove_html_tag
46682,../datasets/news/2018-08-27/6985fa32-38fa-4c03...,20180827081149,6985FA32-38FA-4C03-98E5-103CD6B488D7,元太：H2營收優於H1；全年毛利率拚重回40%之上 ...,\n<P>MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導<...,MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導元太(806...


In [133]:
bert_result = bert_result.drop(['notClick', 'Click'], axis = 1)

In [134]:
bert_result.head(2)

,path,date,news_guid,title,news_content,content_remove_html_tag
0,../datasets/news/2018-05-09/7853913e-eaf7-4c8d...,20180509085858,7853913E-EAF7-4C8D-A21F-2281A02E5DA5,精華Q1 EPS 7元；新產能陸續投產為今年營運添動能 ...,\n<P>MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導<...,MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導隱形眼鏡大廠...
1,../datasets/news/2018-02-01/4cbfc6ab-2526-4b87...,20180201133759,4CBFC6AB-2526-4B87-85BE-7B646A214BA7,環球晶：半導體矽晶圓仍有漲價空間，今年拚逐季成長 ...,\n<P>MoneyDJ新聞 2018-02-01 13:37:59 記者 新聞中心 報導<...,MoneyDJ新聞 2018-02-01 13:37:59 記者 新聞中心 報導半導體矽晶圓...


In [135]:
bert_result_with_user_last_news = pd.concat([particular_user_last_news, bert_result], axis = 0, ignore_index = True)

In [136]:
bert_result_with_user_last_news.head(2)

,path,date,news_guid,title,news_content,content_remove_html_tag
0,../datasets/news/2018-08-27/6985fa32-38fa-4c03...,20180827081149,6985FA32-38FA-4C03-98E5-103CD6B488D7,元太：H2營收優於H1；全年毛利率拚重回40%之上 ...,\n<P>MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導<...,MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導元太(806...
1,../datasets/news/2018-05-09/7853913e-eaf7-4c8d...,20180509085858,7853913E-EAF7-4C8D-A21F-2281A02E5DA5,精華Q1 EPS 7元；新產能陸續投產為今年營運添動能 ...,\n<P>MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導<...,MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導隱形眼鏡大廠...


# Bert

In [137]:
title=[]
content=[]
title_content=[]

for index,row in tqdm_notebook(bert_result_with_user_last_news.iterrows()):
    title.append(row['title'])
    content.append(row['content_remove_html_tag'])
    title_content.append(row['title']+' '+row['content_remove_html_tag'])
    

## remove html tag

In [138]:
# 處理 title 的 html
li = []
for i in range(len(title)):
    li.append(" ".join(title[i].split()))
title = li

In [139]:
# 處理 content 的 html
li = []
for i in range(len(content)):
    li.append(" ".join(content[i].split()))
content = li

In [140]:
# 處理 content 的 html
li = []
for i in range(len(title_content)):
    li.append(" ".join(title_content[i].split()))
title_content = li

In [141]:
sp='\n'
title_s=sp.join(title)
content_s=sp.join(content)
title_content_s=sp.join(title_content)

In [142]:
with open("/project/at081-group38/AT081150_AN/193_ER/title_s.txt", "w") as text_file:
    text_file.write(title_s)

In [143]:
with open("/project/at081-group38/AT081150_AN/193_ER/content_s.txt", "w") as text_file:
    text_file.write(content_s)

In [144]:
with open("/project/at081-group38/AT081150_AN/193_ER/title_content_s.txt", "w") as text_file:
    text_file.write(title_content_s)

#######################################################################################
######################################################################################

layers=-1,-2,-3,-4 \

In [145]:
!python extract_features.py \
  --input_file=/project/at081-group38/AT081193_ER/title_content_s.txt \
  --output_file=tmp/output.jsonl \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=tmp/news_output/model.ckpt-18200.index \
  --layers=-1 \
  --max_seq_length=128 \
  --batch_size=8

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Traceback (most recent call last):
  File "extract_features.py", line 419, in <module>
    tf.app.run()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/platform/app.py", line 125, in run
    _sys.exit(main(argv))
  File "extract_features.py", line 360, in main
    examples = read_examples(FLAGS.input_file)
  File "extract_features.py", line 325, in read_examples
    line = tokenization.convert_to_unicode(reader.readline())
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/lib/io/file_io.py", line 183, in readline
    self._preread_check()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/lib/io/file_io.py", line 85, in _preread_check
    com

In [146]:
import json

data = []
with open('tmp/output.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [147]:
li = []
for i in range(len(data)):
    result = 1 - spatial.distance.cosine(data[0]['features'][0]['layers'][0]['values'], data[i]['features'][0]['layers'][0]['values'])
    li.append(result)

In [148]:
cosine_similarity_bert_feature = pd.DataFrame(li)
cosine_similarity_bert_feature.columns = ['cosine_similarity']
cosine_similarity_bert_feature = cosine_similarity_bert_feature.sort_values(by = ['cosine_similarity'], ascending=False)

In [149]:
final_candidate_bert_feature = bert_result_with_user_last_news.iloc[cosine_similarity_bert_feature.index[1:400],:]

In [150]:
#如果不存在裡面的話，就把加入Validation_data
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_bert_feature[final_candidate_bert_feature.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

目標不存在候選集合當中


In [151]:
# os.chdir("../") # 指定工作路徑
# os.getcwd() # 返回當前工作路徑

In [152]:
## set dictionary (can define yourself)
jieba.set_dictionary('/project/at081-group38/AT081150_AN/193_ER/jieba/dict.txt.big')
stop_words = open('/project/at081-group38/AT081150_AN/193_ER/jieba/stop_words.txt', encoding="utf-8").read().splitlines()

In [153]:
## filter rules
bert_result_with_user_last_news['content_remove_html_tag'] = bert_result_with_user_last_news['content_remove_html_tag'].str.replace('https?:\/\/\S*', '')
bert_result_with_user_last_news['content_remove_html_tag'] = bert_result_with_user_last_news['content_remove_html_tag'].replace('', np.nan)

In [154]:
## remove data
bert_result_with_user_last_news = bert_result_with_user_last_news.dropna()
bert_result_with_user_last_news = bert_result_with_user_last_news.reset_index(drop=True)
bert_result_with_user_last_news['idx'] = bert_result_with_user_last_news.index

In [155]:
bert_result_with_user_last_news.head(2)

,path,date,news_guid,title,news_content,content_remove_html_tag,idx
0,../datasets/news/2018-08-27/6985fa32-38fa-4c03...,20180827081149,6985FA32-38FA-4C03-98E5-103CD6B488D7,元太：H2營收優於H1；全年毛利率拚重回40%之上 ...,\n<P>MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導<...,MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導元太(806...,0
1,../datasets/news/2018-05-09/7853913e-eaf7-4c8d...,20180509085858,7853913E-EAF7-4C8D-A21F-2281A02E5DA5,精華Q1 EPS 7元；新產能陸續投產為今年營運添動能 ...,\n<P>MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導<...,MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導隱形眼鏡大廠...,1


In [156]:
bert_result_with_user_last_news = bert_result_with_user_last_news.drop(['path', 'news_content'], axis = 1)

In [157]:
bert_result_with_user_last_news['title_content'] = bert_result_with_user_last_news[['content_remove_html_tag', 'title']].apply(lambda x: ''.join(x), axis=1)

In [158]:
bert_result_with_user_last_news.columns = ['date', 'news_guid', 'title', 'content', 'idx', 'title_content']

In [159]:
bert_result_with_user_last_news = bert_result_with_user_last_news.reindex(columns=['date', 'news_guid', 'title', 'content',  'title_content', 'idx'])

In [160]:
bert_result_with_user_last_news.head(2)

,date,news_guid,title,content,title_content,idx
0,20180827081149,6985FA32-38FA-4C03-98E5-103CD6B488D7,元太：H2營收優於H1；全年毛利率拚重回40%之上 ...,MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導元太(806...,MoneyDJ新聞 2018-08-27 08:11:49 記者 新聞中心 報導元太(806...,0
1,20180509085858,7853913E-EAF7-4C8D-A21F-2281A02E5DA5,精華Q1 EPS 7元；新產能陸續投產為今年營運添動能 ...,MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導隱形眼鏡大廠...,MoneyDJ新聞 2018-05-09 08:58:58 記者 新聞中心 報導隱形眼鏡大廠...,1


## 先進行斷詞 (選擇 title、 content、 title_content)

In [161]:
# 可選擇 title、 content、 title_content
content_data = bert_result_with_user_last_news['title_content'].tolist()

In [162]:
sentences = []

for i, text in enumerate(tqdm_notebook(content_data)):
    line = []

    for w in jieba.cut(text, cut_all=False):
        
        ## remove stopwords and digits
        ## can define your own rules
        if w not in stop_words and not bool(re.match('[0-9]+', w)):
            line.append(w)

    sentences.append(line)

Building prefix dict from /project/at081-group38/AT081150_AN/193_ER/jieba/dict.txt.big ...
2019-07-04 09:39:00,537: DEBUG: Building prefix dict from /project/at081-group38/AT081150_AN/193_ER/jieba/dict.txt.big ...
Loading model from cache /tmp/jieba.ufc3a5e786f812e3e4ea9d2b74f1db274.cache
2019-07-04 09:39:00,539: DEBUG: Loading model from cache /tmp/jieba.ufc3a5e786f812e3e4ea9d2b74f1db274.cache
Loading model cost 1.217 seconds.
2019-07-04 09:39:01,755: DEBUG: Loading model cost 1.217 seconds.
Prefix dict has been built succesfully.
2019-07-04 09:39:01,756: DEBUG: Prefix dict has been built succesfully.


In [163]:
# print(sentences[0:5])

In [164]:
#######################################################
cut_data = 'content_cutted'
########################################################
## save data as pickle format
with open('/project/at081-group38/AT081150_AN/193_ER/'+ cut_data, "wb") as file:
    pickle.dump(sentences, file)

In [165]:
# 通過 logging.basicConfig 函數對日誌的輸出格式及方式做相關的配置
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

In [166]:
## load 'content_cutted'
with open('/project/at081-group38/AT081150_AN/193_ER/'+ cut_data, 'rb') as file:
    data = pickle.load(file)

## average word vec

In [167]:
# build word2vec
# sg=0 CBOW ; sg=1 skip-gram
model = word2vec.Word2Vec(size=256, min_count=5, window=5, sg=0)

In [168]:
# build vocabulary
model.build_vocab(data)

2019-07-04 09:39:19,647: INFO: collecting all words and their counts
2019-07-04 09:39:19,648: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-04 09:39:19,713: INFO: collected 25371 word types from a corpus of 338503 raw words and 1001 sentences
2019-07-04 09:39:19,713: INFO: Loading a fresh vocabulary
2019-07-04 09:39:19,742: INFO: min_count=5 retains 6695 unique words (26% of original 25371, drops 18676)
2019-07-04 09:39:19,743: INFO: min_count=5 leaves 308303 word corpus (91% of original 338503, drops 30200)
2019-07-04 09:39:19,759: INFO: deleting the raw counts dictionary of 25371 items
2019-07-04 09:39:19,760: INFO: sample=0.001 downsamples 42 most-common words
2019-07-04 09:39:19,760: INFO: downsampling leaves estimated 279954 word corpus (90.8% of prior 308303)
2019-07-04 09:39:19,761: INFO: estimated required memory for 6695 words and 256 dimensions: 17058860 bytes
2019-07-04 09:39:19,774: INFO: resetting layer weights


In [169]:
# train word2vec model ; shuffle data every epoch
for i in range(20):
    random.shuffle(data)
    model.train(data, total_examples=len(data), epochs=1)

2019-07-04 09:39:19,871: INFO: training model with 3 workers on 6695 vocabulary and 256 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-07-04 09:39:20,118: INFO: worker thread finished; awaiting finish of 2 more threads
2019-07-04 09:39:20,122: INFO: worker thread finished; awaiting finish of 1 more threads
2019-07-04 09:39:20,128: INFO: worker thread finished; awaiting finish of 0 more threads
2019-07-04 09:39:20,128: INFO: training on 338503 raw words (280004 effective words) took 0.3s, 1107376 effective words/s
2019-07-04 09:39:20,129: INFO: training model with 3 workers on 6695 vocabulary and 256 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-07-04 09:39:20,375: INFO: worker thread finished; awaiting finish of 2 more threads
2019-07-04 09:39:20,380: INFO: worker thread finished; awaiting finish of 1 more threads
2019-07-04 09:39:20,381: INFO: worker thread finished; awaiting finish of 0 more threads
2019-07-04 09:39:20,381: INFO: training on 338503 

In [170]:
# ## save model
# model.save('/project/at081-group38/AT081193_ER/word2vec_model/CBOW')

In [171]:
# ## load word2vec model
# model = word2vec.Word2Vec.load('/project/at081-group38/AT081193_ER/word2vec_model/CBOW')

In [172]:
## filter words that not in word2vec's vocab
data_filtered = [[w for w in l if w in model.wv] for l in data]

In [173]:
## compute average word vector
avg_vector = []

for l in data_filtered:
    if len(l)==0:
        avg_vector.append(np.array([0]*256))
    else:
        avg_vector.append(np.mean([model.wv[w] for w in l], axis=0))

In [174]:
# ## print result
# avg_vector[0]

In [175]:
# ## save result
# with open('/project/at081-group38/AT081193_ER/word2vec_model/'+cut_data+'avg_vector', 'wb') as file:
#     pickle.dump(avg_vector, file)

In [176]:
# ## load average word2vec result
# with open('/project/at081-group38/AT081193_ER/word2vec_model/'+cut_data+'avg_vector', 'rb') as file:
#     avg_vector = pickle.load(file)

In [177]:
li = []
for i in range(len(avg_vector)):
    result = 1 - spatial.distance.cosine(avg_vector[0], avg_vector[i])
    li.append(result)

In [178]:
cosine_similarity_average_word_vec = pd.DataFrame(li)
cosine_similarity_average_word_vec.columns = ['cosine_similarity']
cosine_similarity_average_word_vec = cosine_similarity_average_word_vec.sort_values(by = ['cosine_similarity'], ascending=False)

In [179]:
final_candidate_average_word_vec = bert_result_with_user_last_news.iloc[cosine_similarity_average_word_vec.index[1:400],:]

In [180]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_average_word_vec[final_candidate_average_word_vec.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

目標不存在候選集合當中


# doc2vec

In [181]:
## create a document id map
sentence_list = []

for i, l in enumerate(data):
    sentence_list.append(doc2vec.LabeledSentence(words=l, tags=[str(i)]))

In [182]:
# ## print result
# sentence_list[0]

In [183]:
## define 轉換器
model = Doc2Vec(size=256, min_count=5, window=4)

In [184]:
## build vocabulary
model.build_vocab(sentence_list)

2019-07-04 09:39:28,440: INFO: collecting all words and their counts
2019-07-04 09:39:28,440: INFO: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-07-04 09:39:28,494: INFO: collected 25371 word types and 1001 unique tags from a corpus of 1001 examples and 338503 words
2019-07-04 09:39:28,494: INFO: Loading a fresh vocabulary
2019-07-04 09:39:28,517: INFO: min_count=5 retains 6695 unique words (26% of original 25371, drops 18676)
2019-07-04 09:39:28,517: INFO: min_count=5 leaves 308303 word corpus (91% of original 338503, drops 30200)
2019-07-04 09:39:28,533: INFO: deleting the raw counts dictionary of 25371 items
2019-07-04 09:39:28,534: INFO: sample=0.001 downsamples 42 most-common words
2019-07-04 09:39:28,534: INFO: downsampling leaves estimated 279954 word corpus (90.8% of prior 308303)
2019-07-04 09:39:28,535: INFO: estimated required memory for 6695 words and 256 dimensions: 18284084 bytes
2019-07-04 09:39:28,548: INFO: resetting layer weights


In [185]:
# train word2vec model ; shuffle data every epoch
for i in range(20):
    random.shuffle(sentence_list)
    model.train(sentence_list, total_examples=len(data), epochs=1)

2019-07-04 09:39:28,647: INFO: training model with 3 workers on 6695 vocabulary and 256 features, using sg=0 hs=0 sample=0.001 negative=5 window=4
2019-07-04 09:39:28,914: INFO: worker thread finished; awaiting finish of 2 more threads
2019-07-04 09:39:28,925: INFO: worker thread finished; awaiting finish of 1 more threads
2019-07-04 09:39:28,929: INFO: worker thread finished; awaiting finish of 0 more threads
2019-07-04 09:39:28,929: INFO: training on 338503 raw words (280713 effective words) took 0.3s, 1008762 effective words/s
2019-07-04 09:39:28,930: INFO: training model with 3 workers on 6695 vocabulary and 256 features, using sg=0 hs=0 sample=0.001 negative=5 window=4
2019-07-04 09:39:29,197: INFO: worker thread finished; awaiting finish of 2 more threads
2019-07-04 09:39:29,209: INFO: worker thread finished; awaiting finish of 1 more threads
2019-07-04 09:39:29,215: INFO: worker thread finished; awaiting finish of 0 more threads
2019-07-04 09:39:29,216: INFO: training on 338503 

In [186]:
# ## print result
# model.docvecs[0]

In [187]:
# ## save model
# model.save('/project/at081-group38/AT081193_ER/word2vec_model/doc2vec')

In [188]:
# ## load word2vec model
# model = word2vec.Word2Vec.load('/project/at081-group38/AT081193_ER/word2vec_model/doc2vec')

In [189]:
li = []
for i in range(len(model.docvecs)):
    result = 1 - spatial.distance.cosine(model.docvecs[0], model.docvecs[i])
    li.append(result)

In [190]:
cosine_similarity_doc2vec = pd.DataFrame(li)
cosine_similarity_doc2vec.columns = ['cosine_similarity']
cosine_similarity_doc2vec = cosine_similarity_doc2vec.sort_values(by = ['cosine_similarity'], ascending=False)

In [191]:
final_candidate_doc2vec = bert_result_with_user_last_news.iloc[cosine_similarity_doc2vec.index[1:400],:]

In [192]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_doc2vec[final_candidate_doc2vec.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

目標不存在候選集合當中


## bag of words

In [193]:
content_data = pd.DataFrame(content_data, columns=['content'])

In [194]:
## word count
## http://blog.csdn.net/gatieme/article/details/43235791 (中文正則表達式)
content_data['word_count'] = content_data['content'].str.count('[a-zA-Z0-9]+') + content_data['content'].str.count('[\u4e00-\u9fff]')

In [195]:
## punctuation count
content_data['punctuation'] = content_data['content'].str.replace('[\w\s]', '')
content_data['punctuation_count'] = content_data['punctuation'].str.len()

In [196]:
## question mark count
content_data['question_count'] = content_data['punctuation'].str.count('[?？]')

In [197]:
## drop punctuation column
content_data = content_data.drop(['punctuation'],axis=1)

In [198]:
# # punctuation 是標點符號
# content_data.head()

In [199]:
# ## compute correlation
# content_data.iloc[:, 1:].corr()

In [200]:
from sklearn.feature_extraction.text import CountVectorizer

In [201]:
## define transformer (轉換器)
vectorizer = CountVectorizer()
count = vectorizer.fit_transform([' '.join(x) for x in data])

In [202]:
## save data as pickle format
with open('/project/at081-group38/AT081150_AN/193_ER/word2vec_model/news_count', "wb") as file:
    pickle.dump([vectorizer, count], file)

In [203]:
## load bag of words result
with open('/project/at081-group38/AT081150_AN/193_ER/word2vec_model/news_count', 'rb') as file:
    _, count = pickle.load(file)

In [204]:
## select top 256 words (counts of document) 
most_count_id = np.array((count > 0).sum(axis=0))[0].argsort()[::-1][:256]

In [205]:
## subset data
count = count[:, most_count_id]

In [206]:
count_data = np.zeros((content_data.shape[0], 256))

In [207]:
## subset bag of words matrix
count_data = count[count_data[:, 0]].toarray()

In [208]:
li = []
for i in range(len(count_data)):
    result = 1 - spatial.distance.cosine(count_data[0], count_data[i])
    li.append(result)

In [209]:
cosine_similarity_bag_of_words = pd.DataFrame(li)
cosine_similarity_bag_of_words.columns = ['cosine_similarity']
cosine_similarity_bag_of_words = cosine_similarity_bag_of_words.sort_values(by = ['cosine_similarity'], ascending=False)

In [210]:
final_candidate_bag_of_words = bert_result_with_user_last_news.iloc[cosine_similarity_bag_of_words.index[1:400],:]

In [211]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_bag_of_words[final_candidate_bag_of_words.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

目標已存在候選集合當中


## TF-IDF

In [212]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [213]:
## define transformer (轉換器)
vectorizer = TfidfVectorizer(norm=None) ## do not do normalize
tfidf = vectorizer.fit_transform([' '.join(x) for x in data])

In [214]:
## save data as pickle format
with open('/project/at081-group38/AT081150_AN/193_ER/word2vec_model/news_tfidf', "wb") as file:
    pickle.dump([vectorizer, tfidf], file)

### select top 10 average tf-idf of words

In [215]:
# create a dictionary: id as key ; word as values
# id2word = {v:k for k, v in vectorizer.vocabulary_.items()}

In [216]:
# ## columnwise average: words tf-idf
# avg = tfidf.sum(axis=0) / (tfidf!=0).sum(axis=0)

# ## set df < 20 as 0
# avg[(tfidf!=0).sum(axis=0)<20] = 0

In [217]:
# avg = np.array(avg)[0]

In [218]:
# ## top 10 tfidf's wordID
# most_avg_id = avg.argsort()[::-1][:10].tolist()
# most_avg_id

In [219]:
# ## print top 10 tf-idf's words
# features = [id2word[i] for i in most_avg_id]
# features

In [220]:
## load tf-idf result
with open('/project/at081-group38/AT081150_AN/193_ER/word2vec_model/news_tfidf', 'rb') as file:
    _, tfidf = pickle.load(file)

In [221]:
## select top 256 words (counts of document) 
most_tfidf_id = np.array((tfidf > 0).sum(axis=0))[0].argsort()[::-1][:256]

In [222]:
## subset data
tfidf = tfidf[:, most_tfidf_id]

In [223]:
tfidf_data = np.zeros((content_data.shape[0], 256))

In [224]:
## subset tf-idf matrix
tfidf_data = tfidf[tfidf_data[:, 0]].toarray()

In [225]:
li = []
for i in range(len(tfidf_data)):
    result = 1 - spatial.distance.cosine(count_data[0], count_data[i])
    li.append(result)

In [226]:
cosine_similarity_TF_IDF = pd.DataFrame(li)
cosine_similarity_TF_IDF.columns = ['cosine_similarity']
cosine_similarity_TF_IDF = cosine_similarity_TF_IDF.sort_values(by = ['cosine_similarity'], ascending=False)

In [227]:
final_candidate_TF_IDF = bert_result_with_user_last_news.iloc[cosine_similarity_TF_IDF.index[1:400],:]

In [228]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_TF_IDF[final_candidate_TF_IDF.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

目標已存在候選集合當中


# Hit rate

In [229]:
# 總篩選篇數約48000筆
total_news_num = 48000
# 後選集合佔總篇數的百分比
percentage = [0.25/100, 0.5/100, 1/100, 1.5/100, 2/100]
# 候選集合數 [120, 240, 480, 720, 960]
n = ( np.array(percentage)*total_news_num ).tolist()
score = [1, 0.9, 0.8, 0.7, 0.6]

In [230]:
li = []
for i in n:
    final_candidate_bert_feature = bert_result_with_user_last_news.iloc[cosine_similarity_bert_feature.index[1:int(i)],:]

    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_bert_feature[final_candidate_bert_feature.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
    
hit_rate_bert = sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_bert

0.6

In [231]:
li = []
for i in n:
    final_candidate_average_word_vec = bert_result_with_user_last_news.iloc[cosine_similarity_average_word_vec.index[1:int(i)],:]
    
    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_average_word_vec[final_candidate_average_word_vec.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_average_word_vec= sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_average_word_vec

0.8

In [232]:
li = []
for i in n:
    final_candidate_doc2vec = bert_result_with_user_last_news.iloc[cosine_similarity_doc2vec.index[1:int(i)],:]
    
    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_doc2vec[final_candidate_doc2vec.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_doc2vec= sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_doc2vec   

0.8

In [233]:
li = []
for i in n:
    final_candidate_bag_of_words = bert_result_with_user_last_news.iloc[cosine_similarity_bag_of_words.index[1:int(i)],:]
    
    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_bag_of_words[final_candidate_bag_of_words.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_bag_of_words = sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_bag_of_words         

0.8

In [234]:
li = []
for i in n:
    final_candidate_TF_IDF = bert_result_with_user_last_news.iloc[cosine_similarity_TF_IDF.index[1:int(i)],:]

    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_TF_IDF[final_candidate_TF_IDF.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_TF_IDF = sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_TF_IDF           

0.8